In [10]:
from msclap import CLAP
import torch.nn.functional as F
import gradio as gr

In [11]:
# Define classes for zero-shot
# Should be in lower case and can be more than one word
classes = ['clap','clapping','snapping', 'snap', 'smack', 'talking', 'speaking', 'voice']
ground_truth = ['clapping']
# Add prompt
prompt = 'this is a sound of '
class_prompts = [prompt + x for x in classes]
file = ['tutorials/audio-samples/angelo-clap1.mp3']

In [19]:
def print_result(strlist):
    #Print the results
    print("Ground Truth: {}".format(ground_truth))
    print("Top predictions:\n")
    print(strlist)
    

In [23]:
def classify(audio_files):
    # Load and initialize CLAP
    # Setting use_cuda = True will load the model on a GPU using CUDA  
    clap_model = CLAP(version= '2023', use_cuda= False)
    
    # compute text embeddings from natural text
    text_embeddings = clap_model.get_text_embeddings(class_prompts)
    
    # compute the audio embeddings from an audio file
    audio_embeddings = clap_model.get_audio_embeddings(audio_files, resample=True)
    
    # compute the similarity between audio_embeddings and text_embeddings
    similarity = clap_model.compute_similarity(audio_embeddings, text_embeddings)

    similarity = F.softmax(similarity, dim=1)
    values, indices = similarity[0].topk(5)
    
    print_result(values, indices)
    
    return result_to_str(values, indices)
    
    

In [28]:
test = classify(file)

Ground Truth: ['clapping']
Top predictions:

['           smack: 83.25%', '        snapping: 9.13%', '            snap: 4.36%', '        clapping: 2.11%', '            clap: 1.03%']


In [25]:
print(test)

['           smack: 83.25%', '        snapping: 9.13%', '            snap: 4.36%', '        clapping: 2.11%', '            clap: 1.03%']


In [3]:
def transcribe(audio):
    output = classify(audio,labels=["snap","snapping","clap","clapping","talking","voice"])["text"]
    return output

In [25]:
gr.Interface(
    fn=classify,
    inputs=[gr.Audio(sources="microphone",type="filepath")],
    outputs=["textbox"],
    live=True).launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/lib/python3.11/site-packages/gradio/queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/lib/python3.11/site-packages/gradio/route_utils.py", line 253, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1695, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1235, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get